# Ingest data into Microsoft Fabric lakehouse using Spark



## **Customer Churn Prediction**
Churn prediction modeling techniques attempt to understand the precise customer behaviors and attributes which can help you to predict which customers are more likely to unsubscribe or leave your services

![Customer Churn](https://az712634.vo.msecnd.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/ea949e1c6b2d4ed99bd725b20380dca1/effcb4cc2dc24d52999d149f27aa0f26/image)

### Sample Dataset
The sample dataset contains churn status of 10,000 customers along with 14 attributes that include credit score, geographical location (Germany, France, Spain), gender (male, female), age, tenure (years of being bank's customer), account balance, estimated salary, number of products that a customer has purchased through the bank, credit card status (whether a customer has a credit card or not), and active member status (whether an active bank's customer or not).

The dataset also includes columns such as row number, customer ID, and customer surname that should have no impact on customer's decision to leave the bank. The event that defines the customer's churn is the closing of the customer's bank account, therefore, the column exit in the dataset refers to customer's abandonment. Since you don't have much context about these attributes, you'll proceed without having background information about the dataset. Your aim is to understand how these attributes contribute to the exit status.

Out of the 10,000 customers, only 2037 customers (around 20%) have left the bank. Therefore, given the class imbalance ratio, we recommend generating synthetic data. Moreover, confusion matrix accuracy may not be meaningful for imbalanced classification and it might be better to also measure the accuracy using the Area Under the Precision-Recall Curve (AUPRC).


### Ingest the data

* Make sure you add a lakehouse to the notebook before running it. Failure to do so will result in an error.

In [1]:
# Azure storage access info for the sample churn.csv data
storage_account = "synapseaisolutionsa"
container = "public"
dir_name = "bankcustomerchurn"
file_name = "churn.csv"

wasbs_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/{dir_name}/{file_name}"

df = spark.read.format("csv").option("header", True).option("inferSchema", True).load(wasbs_path).cache()


StatementMeta(, 2dd1d575-6df3-474d-a031-983e1cd91b55, 3, Finished, Available, Finished)

In [7]:
df.count()

StatementMeta(, 11eb49d6-6e4f-46d1-a9f7-cafaa9f0b880, 9, Finished, Available)

10000

### Enable Vorder and Optimized Delta Write*

**VOrder**: Fabric includes Microsoft's VOrder engine. VOrder writer optimizes the Delta Lake parquet files resulting in 3x-4x compression improvement and up to 10x performance acceleration over Delta Lake files not optimized using VOrder while still maintaining full Delta Lake and PARQUET format compliance.

**Optimize write**: Spark in Fabric includes an Optimize Write feature that reduces the number of files written and targets to increase individual file size of the written data. It dynamically optimizes files during write operations generating files with a default 128 MB size. The target file size may be changed per workload requirements using configurations.
These configs can be applied at a session level(as spark.conf.set in a notebook cell) as demonstrated in the following code cell, or at workspace level which is applied automatically to all spark sessions created in the workspace. The workspace level Apache Spark configuration can be set at:

**Note**: These are now enabled **by default**. For more details refer to https://learn.microsoft.com/en-us/fabric/data-engineering/delta-optimization-and-v-order?tabs=pyspark


In [8]:
# spark.conf.set("sprk.sql.parquet.vorder.enabled", "true") # VOrder write is enabled by default
# spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true") # automatic delta optimized write is enabled by default

StatementMeta(, 11eb49d6-6e4f-46d1-a9f7-cafaa9f0b880, 10, Finished, Available)

### Write Spark dataframe to lakehouse delta table

In [2]:
table_name = "churn_raw"
df.write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")

StatementMeta(, 2dd1d575-6df3-474d-a031-983e1cd91b55, 4, Finished, Available, Finished)

Spark dataframe saved to delta table: churn_raw
